## 数据预处理

In [2]:
import os
import cv2
import h5py
import numpy as np
import pandas as pd
from keras.applications import Xception, xception
from keras.models import Model
from keras.layers import Dropout, Dense, Input, Lambda
from keras.callbacks import Callback
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
data_path_train = '../dogs-vs-cats-dataset/train'
data_path_test = '../dogs-vs-cats-dataset/test'
image_names_train = os.listdir(data_path_train)
image_names_test = os.listdir(data_path_test)
input_shape = (299, 299, 3)
labels = []
trains = []
tests = []

# 处理标准的训练数据
for i in tqdm(range(len(image_names_train))):
    image_name = image_names_train[i]
    image_path = os.path.join(data_path_train, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read train image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    trains.append(image[:, :, ::-1])
    # cat: 0, dog: 1
    category = 1 if 'dog' in image_name else 0
    labels.append(category)


# 处理标准的测试数据
for i in tqdm(range(len(image_names_test))):
    image_name = image_names_test[i]
    image_path = os.path.join(data_path_test, image_name)
    image = cv2.imread(image_path)
    if image is None:
        print('Read test image failed:', image_path)
        continue
    image = cv2.resize(image, (input_shape[0], input_shape[1]))
    tests.append(image[:, :, ::-1])
    
    
trains = np.array(trains)
labels = np.array(labels)
tests = np.array(tests)

print('Training data size: %d' % len(trains))
print('Label size: %d' % len(labels))
print('Testing data size: %d' % len(tests))

100%|██████████| 12500/12500 [00:30<00:00, 415.64it/s]


Training data size: 25000
Label size: 25000
Testing data size: 12500


## 特征提取

In [ ]:
x = Input(shape=input_shape)
x = Lambda(xception.preprocess_input)(x)
model = Xception(input_tensor=x, input_shape=input_shape, weights='imagenet', include_top=False, pooling='avg')
bottleneck_features_train = model.predict(trains, batch_size=128)
bottleneck_features_test = model.predict(tests, batch_size=128)

with h5py.File("bottleneck_features.h5", 'w') as h:
    h.create_dataset('trains', data=bottleneck_features_train)
    h.create_dataset('labels', data=labels)
    h.create_dataset('tests', data=bottleneck_features_test)
    h.create_dataset('test_imgs', data=image_names_test)

print('bottleneck features have been wrote to bottleneck_features.h5')

## 构建模型

In [42]:
with h5py.File('bottleneck_features.h5','r') as h:
    X_train = np.array(h['trains'])
    y_train = np.array(h['labels'])
    X_test = np.array(h['tests'])
    test_imgs = np.array(h['test_imgs'])

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, shuffle=True, test_size=0.2, random_state=2018)

x = Input(shape=(X_train.shape[1],))
y = Dropout(0.2)(x)
y = Dense(1, activation='sigmoid')(y)
model = Model(x, y)

model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

print('Model ready!')

Model ready!


## 训练

### 预测函数

In [43]:
def predict_func(mod, file_name):
    y_pred = mod.predict(X_test, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=0.995)

    df = pd.read_csv("sample_submission.csv")

    for i in range(len(test_imgs)):
        image_name = test_imgs[i]
        index = int(str.split(image_name, '.')[0]) - 1
        df.iat[index, 1] = y_pred[i]

    df.to_csv(os.path.join('./predict-csv', file_name), index=None)
    print('The prediction result has been wrote to: ', file_name)

### 回调函数

In [44]:
class LossCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        predict_func(self.model, 'predict' + '_epoch' + str(epoch + 1) + '.csv')

### 模型训练与优化

In [45]:

model.fit(
    x=X_train,
    y=y_train,
    batch_size=128,
    epochs=10,
    validation_data=(X_val, y_val),
    callbacks=[LossCallback()]
)


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
12500/12500 [==============================] - 0s 40us/step
The prediction result has been wrote to:  predict_epoch1.csv
Epoch 2/10
12500/12500 [==============================] - 0s 30us/step
The prediction result has been wrote to:  predict_epoch2.csv
Epoch 3/10
12500/12500 [==============================] - 0s 30us/step
The prediction result has been wrote to:  predict_epoch3.csv
Epoch 4/10
12500/12500 [==============================] - 0s 30us/step
The prediction result has been wrote to:  predict_epoch4.csv
Epoch 5/10
12500/12500 [==============================] - 0s 29us/step
The prediction result has been wrote to:  predict_epoch5.csv
Epoch 6/10
12500/12500 [==============================] - 0s 30us/step
The prediction result has been wrote to:  predict_epoch6.csv
Epoch 7/10
12500/12500 [==============================] - 0s 30us/step
The prediction result has been wrote to:  predict_epoch7.csv
Epoch 8/10
12500/12500 [===

## 预测

In [58]:
predict_func(model, 'predict.csv')

12500/12500 [==============================] - 1s 82us/step
The prediction result has been wrote to predict.csv
